In [1]:
import pandas as pd
import numpy as np
import math
from shapely.geometry import Polygon, Point
import shapefile
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("Clean_Process_data"), '..', 'functions/data_transforming')))


In [2]:
reader= shapefile.Reader('../data/raw_data/zip_codes/geo_export_37348335-ac06-4ade-bf6d-453d393d539d.shp')

In [3]:
print dict((d[0],d[1:]) for d in reader.fields[1:])

{'po_name': ['C', 254, 0], 'state': ['C', 254, 0], 'length': ['N', 33, 31], 'zip': ['C', 254, 0], 'area': ['N', 33, 31]}


In [4]:
df = pd.DataFrame(columns=['zip','shape'])
fields = [field[0] for field in reader.fields[1:]]
for i,feature in enumerate(reader.shapeRecords()):
    geom = feature.shape.__geo_interface__
    atr = dict(zip(fields, feature.record))
    try:
        result = pd.DataFrame({'zip':atr['zip'],'shape':Polygon(list(geom['coordinates'][0]))},index=[i])
        df = df.append(result)
    except:
        print'error'

error
error
error
error
error
error
error
error
error


In [5]:
df = df[df["zip"] != '94130']
df = df[df["zip"] != '94129']
df = df[df["zip"] != '94128']
df = df[df["zip"].isin([
            '94102', '94103', '94104', '94105','94107',
  '94108', '94109', '94110', '94111', '94112',
  '94114', '94115','94116', '94117', '94118',
  '94121','94122', '94123', '94124',  '94127',
  '94131', '94132', '94133', '94134', '94158'])]

##missing 94107 and 94158

In [6]:
import random

def get_random_point_in_polygon(poly):
    a = []
    (minx, miny, maxx, maxy) = poly.bounds
    ct=0
    while ct <5:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            a.append(p)
            ct = ct+1
    return a



In [7]:
## every zipcode get 10 random points
zipcode= []
shape = []
for i, row in df.iterrows():
    shape.append(get_random_point_in_polygon(row['shape']))
    zipcode.append(row['zip'])

In [8]:
w = shapefile.Writer(shapefile.POINT)
w.autoBalance = 1 #ensures gemoetry and attributes match
w.field('X','F',10,8)
w.field('Y','F',10,8)
w.field('ID','N')

for i,shapes in enumerate(shape):
    print zipcode[i]
    for s in shapes:   
        a =list(s.coords)
        w.point(a[0][1],a[0][0]) #write the geometry
        w.record(a[0][1],a[0][0],int(zipcode[i])) #write the attributes

#Save shapefile
#w.save('point')




94123
94109
94111
94108
94115
94105
94104
94118
94121
94102
94103
94117
94122
94114
94110
94131
94116
94124
94127
94132
94112
94134


In [9]:
##94158
code158 = [(37.768920, -122.388867),(37.774823, -122.394446),(37.771812, -122.399453),(37.769287, -122.395749),(37.765080, -122.389076)]
for s in code158:

    w.point(s[0],s[1]) #write the geometry
    w.record(s[0],s[1],94158) #write the attributes



In [10]:
##94107
code107 = [(37.779797, -122.396510),(37.775523, -122.399170),(37.780746, -122.401488),(37.784545, -122.395308),(37.761651, -122.397715),(37.759870, -122.400224),(37.753945, -122.389292)]
for s in code107:
    w.point(s[0],s[1]) #write the geometry
    w.record(s[0],s[1],94107) #write the attributes


In [11]:
##94133 
code133= [(37.804209, -122.413568),(37.802310, -122.408419),(37.807125, -122.412109),(37.799665, -122.404814),(37.795867, -122.413225)]
for s in code133:
    w.point(s[0],s[1]) #write the geometry
    w.record(s[0],s[1],94133) #write the attributes


In [12]:
w.save('point')